In [2]:
import cv2 as cv2
import numpy as np
import os

In [3]:
# === Resolve paths relative to this script ===
base_dir = os.getcwd()
model_dir = os.path.join(base_dir, 'Model')
print(model_dir)

C:\Users\Darshan\Computer Vision\Model


In [4]:
prototxt = os.path.join(model_dir, 'colorization_deploy_v2.prototxt')
model = os.path.join(model_dir, 'colorization_release_v2.caffemodel')
pts_npy = os.path.join(model_dir, 'pts_in_hull.npy')

In [5]:
net = cv2.dnn.readNetFromCaffe(prototxt, model)
pts = np.load(pts_npy)

In [6]:
# Add cluster centers as 1x1 convolution kernel to model
class8_ab = net.getLayerId('class8_ab')
conv8_313_rh = net.getLayerId('conv8_313_rh')
net.getLayer(class8_ab).blobs = [pts.transpose().reshape(2, 313, 1, 1)]
net.getLayer(conv8_313_rh).blobs = [np.full([1, 313], 2.606, dtype='float32')]

In [7]:
# === Load and Display Grayscale Image ===
image_path = "C:\\Users\\Darshan\\Downloads\\Computer Vision\\Monster1.jpg"
img = cv2.imread(image_path)

In [8]:
if img is None:
    print("❌ Failed to load image.")
    exit()

In [9]:
cv2.imshow("Original Image", img)

In [10]:
# Convert to Lab and extract L channel
img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
img_l = img_lab[:, :, 0]

In [11]:
# Resize image to network input size
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img_rgb, (224, 224))
img_lab_rs = cv2.cvtColor(img_resized, cv2.COLOR_RGB2Lab)
l_rs = img_lab_rs[:, :, 0]
l_rs -= 50  # Center the L channel to zero mean

In [12]:
# Prepare input blob and run forward pass
blob = cv2.dnn.blobFromImage(l_rs, scalefactor=1.0, size=(224, 224), mean=(50,))
net.setInput(blob)
ab_dec = net.forward()[0, :, :, :].transpose((1, 2, 0))

In [13]:
# Resize the ab output to original image size
ab_dec_us = cv2.resize(ab_dec, (img.shape[1], img.shape[0]))

In [14]:
# Combine with original L channel
lab_out = np.concatenate((img_l[:, :, np.newaxis], ab_dec_us), axis=2)
bgr_out = cv2.cvtColor(lab_out.astype(np.uint8), cv2.COLOR_Lab2BGR)

C:\Users\Darshan\AppData\Local\Temp\ipykernel_4332\656122446.py:3: RuntimeWarning: invalid value encountered in cast
  bgr_out = cv2.cvtColor(lab_out.astype(np.uint8), cv2.COLOR_Lab2BGR)


In [15]:
# === Show Final Colorized Image ===
cv2.imshow("Colorized Image", bgr_out)
cv2.waitKey(0)
cv2.destroyAllWindows()